In [5]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

# Paths to dataset directories
dataset_folder = 'dataset'
models_folder = os.path.join(dataset_folder, 'models')
scenes_folder = os.path.join(dataset_folder, 'scenes')

# Load images
def load_images(folder, prefix, count):
    images = []
    for i in range(1, count + 1):
        path = os.path.join(folder, f'{prefix}{i}.png')
        image = cv2.imread(path)
        if image is not None:
            images.append(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    return images

ref_images = load_images(models_folder, 'ref', 14)
scene_images = load_images(scenes_folder, 'scene', 5)

In [6]:
# Denoising function
def Denoise(image):
    # Convert to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_RGB2LAB)
    l, a, b = cv2.split(lab)
    # Apply CLAHE to L-channel
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8,8))
    cl = clahe.apply(l)
    limg = cv2.merge((cl, a, b))
    image_enhanced = cv2.cvtColor(limg, cv2.COLOR_LAB2RGB)
    
    # Apply advanced denoising
    image_denoised = cv2.fastNlMeansDenoisingColored(image_enhanced, None, 10, 10, 7, 21)
    
    return image_denoised

scene_images_denoised = [Denoise(cv2.cvtColor(img, cv2.COLOR_RGB2BGR)) for img in scene_images]
scene_images_denoised = [cv2.cvtColor(img, cv2.COLOR_BGR2RGB) for img in scene_images_denoised]

In [7]:
# Initialize SIFT detector
sift = cv2.SIFT_create(nfeatures=10000)

# Function to detect and match features using SIFT and FLANN
def detect_and_match_features(ref_image, scene_image):
    # Detect and compute SIFT features
    kp1, des1 = sift.detectAndCompute(ref_image, None)
    kp2, des2 = sift.detectAndCompute(scene_image, None)
    
    # FLANN based matcher
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
    search_params = dict(checks=200)
    
    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)
    
    # Apply ratio test
    good_matches = []
    for m, n in matches:
        if m.distance < 0.6 * n.distance:
            good_matches.append(m)
    
    # Extract matched keypoints
    points1 = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 2)
    points2 = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 2)
    
    # Apply RANSAC to filter out outliers
    if len(points1) >= 4:  # Minimum number of points needed to compute homography
        H, mask = cv2.findHomography(points1, points2, cv2.RANSAC, 3.0)
        matches_mask = mask.ravel().tolist()
    else:
        H, matches_mask = None, []
    
    return kp1, kp2, good_matches, matches_mask, H

# Function to find bounding boxes for matched features
def find_bounding_boxes(ref_image, scene_image, H):
    h, w = ref_image.shape[:2]
    
    # Get bounding box corners
    corners = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1, 1, 2)
    
    if H is not None:
        transformed_corners = cv2.perspectiveTransform(corners, H)
    else:
        transformed_corners = np.zeros((4, 1, 2), dtype=np.float32)
    
    return transformed_corners

# Function to check if bounding box contains sufficient context
def is_valid_bounding_box(bbox, scene_image):
    x_coords = bbox[:, 0, 0]
    y_coords = bbox[:, 0, 1]
    
    if (np.min(x_coords) >= 0 and np.max(x_coords) < scene_image.shape[1] and
        np.min(y_coords) >= 0 and np.max(y_coords) < scene_image.shape[0]):
        area = cv2.contourArea(bbox)
        return area > 5000  # Minimum area threshold
    return False

# Function to calculate bounding box dimensions and center position
def calculate_bbox_details(bbox):
    x_coords = bbox[:, 0, 0]
    y_coords = bbox[:, 0, 1]
    width = int(np.max(x_coords) - np.min(x_coords))
    height = int(np.max(y_coords) - np.min(y_coords))
    center_x = int(np.min(x_coords) + width // 2)
    center_y = int(np.min(y_coords) + height // 2)
    return width, height, (center_x, center_y)

In [10]:
# Process each reference image against each scene image and output the results
for i, ref_image in enumerate(ref_images, 1):
    instance_details = []
    instance_count = 0
    for j, scene_image in enumerate(scene_images_denoised, 1):
        kp1, kp2, good_matches, matches_mask, H = detect_and_match_features(ref_image, scene_image)
        
        bounding_box_valid = False
        if len(good_matches) >= 10 and H is not None:
            bounding_box = find_bounding_boxes(ref_image, scene_image, H)
            bounding_box_valid = is_valid_bounding_box(bounding_box, scene_image)
        
        # If not enough matches or invalid bounding box, prepare a dummy output
        if not bounding_box_valid:
            bounding_box = np.array([[[0, 0]], [[0, 0]], [[0, 0]], [[0, 0]]])
            print(f'  Invalid bounding box or not enough matches for Scene {j} with Reference Image {i}')
        else:
            # Calculate bounding box dimensions and center position
            width, height, center = calculate_bbox_details(bounding_box)
            instance_count += 1
            instance_details.append(f'  Instance {instance_count} {{position: {center}, width: {width}px, height: {height}px}}')
        
        # Draw bounding box on scene image
        scene_image_with_box = scene_image.copy()
        bounding_box = np.int32(bounding_box)
        cv2.polylines(scene_image_with_box, [bounding_box], True, (0, 255, 0), 3, cv2.LINE_AA)
        
        # Draw matched keypoints
        draw_params = dict(matchColor = (0,255,0), # draw matches in green color
                           singlePointColor = None,
                           matchesMask = matches_mask if bounding_box_valid else None, # draw only inliers
                           flags = 2)
        img_matches = cv2.drawMatches(ref_image, kp1, scene_image, kp2, good_matches, None, **draw_params)
        
        # # Display matches and bounding box
        # plt.figure(figsize=(20, 10))
        
        # plt.subplot(1, 2, 1)
        # plt.imshow(img_matches)
        # plt.title(f'Matches for Scene {j}')
        # plt.axis('off')
        
        # plt.subplot(1, 2, 2)
        # plt.imshow(scene_image_with_box)
        # plt.title(f'Bounding Box for Scene {j}')
        # plt.axis('off')
        
        # plt.show()
    
    # Print the summary for the current product
    print(f'Product {i - 1} – {instance_count} instance(s) found.')
    for detail in instance_details:
        print(detail)

  Invalid bounding box or not enough matches for Scene 2 with Reference Image 1
  Invalid bounding box or not enough matches for Scene 3 with Reference Image 1
  Invalid bounding box or not enough matches for Scene 5 with Reference Image 1
Product 0 – 2 instance(s) found.
  Instance 1 {position: (410, 538), width: 801px, height: 1049px}
  Instance 2 {position: (887, 911), width: 380px, height: 516px}
  Invalid bounding box or not enough matches for Scene 2 with Reference Image 2
  Invalid bounding box or not enough matches for Scene 3 with Reference Image 2
  Invalid bounding box or not enough matches for Scene 4 with Reference Image 2
  Invalid bounding box or not enough matches for Scene 5 with Reference Image 2
Product 1 – 1 instance(s) found.
  Instance 1 {position: (1246, 532), width: 802px, height: 1049px}
  Invalid bounding box or not enough matches for Scene 1 with Reference Image 3
  Invalid bounding box or not enough matches for Scene 4 with Reference Image 3
  Invalid boundi